In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests 
from bs4 import BeautifulSoup
from openai import OpenAI
from twilio.rest import Client
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(dotenv_path="/Users/jt/Documents/GitHub/AI_lunch_box_recommender_system/.env") # CHANGE THE PATH HERE IF YOU CANT LOAD .env
  # Load .env file into os.environ

# Global variables
url = "https://www.ica.se/erbjudanden/ica-supermarket-aptiten-1003988/"
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
client_twilio = Client(os.environ["account_sid"], os.environ["auth_token"])

def scrape_your_local_ica(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)

    # clicking agree on cookie button
    agree_button_selector = "button#onetrust-accept-btn-handler"
    WebDriverWait(driver, 3)
    element = driver.find_element(By.CSS_SELECTOR, agree_button_selector)
    element.click()

    # scroll to bottom of page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # generate a soup
    soup = BeautifulSoup(driver.page_source)
    key = "article > div.offer-card__details-container"
    containers = soup.select(key)
    offers = [i.text.replace("\xa0kr. ", "") for i in containers]
    offers = [i.split(". ")[:2] for i in offers]
    return offers

def send_text(from_number, to_number, text_message):
    message = client_twilio.messages.create(
        body=text_message,
        from_=from_number,
        to=to_number
    )

def chat_gpt_bot(client, instructions, prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": prompt},
        ]
    )
    return completion.choices[0].message.content

# prompt instructions for chat gpt bot
role = "Du är en kreativ kock som är väldigt duktig på att rekommendera enkla maträtter baserat på tillgängliga ingredienser."
instructions = "Jag vill att du rekommenderar mig maträtter som jag kan laga till middag och sedan göra matlåda av. Du får en lista med veckans rabatterade priser från en matbutik. Du ska använda listan för att skapa 3 enkla recept. Du får också lägga till basvaror (pasta, ris, potatis, lök etc.), även om de inte är med på listan.\nJag vill att du genererar 3 rätter i rubrikform och under varje rubrik/rätt så ska det finnas en lista med ingredienser för 4 portioner. Det är viktigt att enheterna är i enlighet med det metriska systemet. Under listan med ingredienser ska du beskriva väldigt kort hur man tillagar rätten."

# scrape weekly offers from ICA 
context = scrape_your_local_ica(url)

# generate recipies from ICA offers
completion = chat_gpt_bot(client, role, instructions+str(context))

# generate shopping list from recipies 
shopping_lst = chat_gpt_bot(client, "Hjälpreda", "Generera en inköpslista i punktform med sammanlagda enheter av följande information: "+completion)

#send text from shopping list
#send_text(os.environ["from_number"], os.environ["to_number"], shopping_lst)

# write recipies to txt file
with open("weekly.recipies.txt", "w") as file:
    file.write(completion)
    file.close()

# write shopping list to file (if twilio doesn't work for you)
with open("shopping_list.txt", "w") as file:
    file.write(shopping_lst)
    file.close()